In [1]:
import glob
import json
from os import listdir
from os.path import isfile, join, exists, isdir, abspath

import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

IMAGE_DIM = 224   # required/default image dimensionality

def load_images(image_paths, image_size, verbose=True):
   
    loaded_images = []
    loaded_image_paths = []

    if isdir(image_paths):
        parent = abspath(image_paths)
        image_paths = [join(parent, f) for f in listdir(image_paths) if isfile(join(parent, f))]
    elif isfile(image_paths):
        image_paths = [image_paths]

    for img_path in image_paths:
        try:
            if verbose:
                print(img_path)
            image = keras.preprocessing.image.load_img(img_path, target_size=image_size)
            image = keras.preprocessing.image.img_to_array(image)
            image /= 255
            loaded_images.append(image)
            loaded_image_paths.append(img_path)
        except Exception as ex:
            print("Image Load Failure: ", img_path, ex)
    
    return np.asarray(loaded_images), loaded_image_paths


def load_model(model_path):
    if model_path is None or not exists(model_path):
    	raise ValueError("saved_model_path must be the valid directory of a saved model to load.")
    
    # model = tf.keras.models.load_model(model_path)
    model = tf.keras.models.load_model(model_path, custom_objects={'KerasLayer':hub.KerasLayer})
    # model.summary()
    #print(model.summary())
    return model


def classify(model, input_paths, image_dim=IMAGE_DIM):
    """ Classify given a model, input paths (could be single string), and image dimensionality...."""
    images, image_paths = load_images(input_paths, (image_dim, image_dim))
    probs = classify_nd(model, images)
    return dict(zip(image_paths, probs))


def classify_nd(model, nd_images):
    """ Classify given a model, image array (numpy)...."""

    model_preds = model.predict(nd_images)
    # preds = np.argsort(model_preds, axis = 1).tolist()
    
    categories = ['drawings', 'hentai', 'neutral', 'porn', 'sexy']

    probs = []
    for i, single_preds in enumerate(model_preds):
        single_probs = {}
        for j, pred in enumerate(single_preds):
            single_probs[categories[j]] = float(pred)
        probs.append(single_probs)
    return probs


def main():   
    img_paths  = glob.glob('image_data/*.jpg')
    predictions = []
    print(img_paths)
    for img_path in img_paths:
        
        model = load_model("Nudity-Detection-Model.h5")
        model.compile()
        image_preds = classify(model, img_path, IMAGE_DIM)
        predictions.append(image_preds)
    
    print(predictions)


if __name__ == "__main__":
	main()



['image_data\\10.jpg', 'image_data\\11116.jpg', 'image_data\\cock.jpg', 'image_data\\gun.jpg']


image_data\10.jpg


image_data\11116.jpg


image_data\cock.jpg


image_data\gun.jpg
[{'image_data\\10.jpg': {'drawings': 0.03408496081829071, 'hentai': 0.030753731727600098, 'neutral': 0.05647759139537811, 'porn': 0.8606960773468018, 'sexy': 0.017987744882702827}}, {'image_data\\11116.jpg': {'drawings': 0.026670880615711212, 'hentai': 0.02497580088675022, 'neutral': 0.8212873935699463, 'porn': 0.10139939934015274, 'sexy': 0.025666484609246254}}, {'image_data\\cock.jpg': {'drawings': 0.06420043855905533, 'hentai': 0.01737922802567482, 'neutral': 0.8439819812774658, 'porn': 0.027503060176968575, 'sexy': 0.04693525284528732}}, {'image_data\\gun.jpg': {'drawings': 0.011121034622192383, 'hentai': 0.010665267705917358, 'neutral': 0.9495856165885925, 'porn': 0.004424175247550011, 'sexy': 0.024203896522521973}}]
